In [61]:
import pandas as pd
import mysql.connector
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [62]:
load_dotenv()

password = os.getenv('PASSWORD')
username = os.getenv('USERNAME')
host = os.getenv('HOST')
port = os.getenv('PORT')
netflix_csv = os.getenv('NETFLIX_DIR')
amazon_csv = os.getenv('AMAZON_DIR')
netflix_file = os.getenv('NETFLIX_FILE')
amazon_file = os.getenv('AMAZON_FILE')

df_amazon = pd.read_csv(amazon_csv+amazon_file)
df_netflix = pd.read_csv(netflix_csv+netflix_file)

In [63]:
engine = create_engine('mysql+mysqlconnector://' + username + ':' + password + '@' + host + ':' + port + '/shows')

In [21]:
joined_df_cast = pd.merge(df_amazon, df_netflix, on='cast', how='outer')
joined_df_director = pd.merge(df_amazon, df_netflix, on='director', how='outer')
joined_df_type = pd.merge(df_amazon, df_netflix, on='type', how='outer')

In [11]:
joined_df_cast['cast'] = joined_df_cast['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)

joined_df_cast['cast'] = joined_df_cast['cast'].apply(lambda x: x if isinstance(x, list) else str(x).split(','))
exploded_df_all = joined_df_cast.explode('cast')

exploded_df_all['cast'] = exploded_df_all['cast'].str.strip()

exploded_df_all = exploded_df_all.drop_duplicates(subset=['cast'])

exploded_df_all = exploded_df_all.reset_index(drop=True)

new_df_cast = pd.DataFrame({ 
    'Name': exploded_df_all['cast']
})

In [12]:
#new_df_cast.to_sql('casting', con=engine, if_exists='append', index=False)

60762

In [13]:
joined_df_director['director'] = joined_df_director['director'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)

joined_df_director['director'] = joined_df_director['director'].apply(lambda x: x if isinstance(x, list) else str(x).split(','))
exploded_df_all = joined_df_director.explode('director')

exploded_df_all['director'] = exploded_df_all['director'].str.strip()

exploded_df_all = exploded_df_all.drop_duplicates(subset=['director'])

exploded_df_all = exploded_df_all.reset_index(drop=True)

new_df_director = pd.DataFrame({
    'Name': exploded_df_all['director']
})

new_df_director = new_df_director[new_df_director['Name'].apply(lambda x: len(x) if isinstance(x, str) else 0) <= 255]

In [14]:
#new_df_director.to_sql('director', con=engine, if_exists='append', index=False)

10488

In [17]:
joined_df_type = joined_df_type.drop_duplicates(subset=['type'])

joined_df_type.head()

joined_df_type = joined_df_type.reset_index(drop=True)

new_df_type = pd.DataFrame({ 
    'Type': joined_df_type['type']
})

In [18]:
#new_df_type.to_sql('category', con=engine, if_exists='append', index=False)

2

In [ ]:
new_df_show_amazon = pd.DataFrame({ 
    'Title': df_amazon['title'],
    'Country': df_amazon['country'],
    'Release_Year': df_amazon['release_year'],
    'Duration': df_amazon['duration'],
    'Rating': df_amazon['rating']
})

new_df_show_amazon.head()

In [ ]:
new_df_show_netflix = pd.DataFrame({ 
    'Title': df_netflix['title'],
    'Country': df_netflix['country'],
    'Release_Year': df_netflix['release_year'],
    'Duration': df_netflix['duration'],
    'Rating': df_netflix['rating']
})

new_df_show_netflix.head()

In [107]:
#new_df_show_amazon.to_sql('shows', con=engine, if_exists='append', index=False)

9668

In [108]:
#new_df_show_netflix.to_sql('shows', con=engine, if_exists='append', index=False)

8807

In [131]:
shows_df_amazon = df_amazon[['title', 'type', 'cast', 'director']]

In [132]:
type_mapping_df = pd.read_sql('SELECT ID, Type FROM category', engine)
casting_mapping_df = pd.read_sql('SELECT ID, Name FROM casting', engine)
director_mapping_df = pd.read_sql('SELECT ID, Name FROM director', engine)
show_mapping_df = pd.read_sql('SELECT ID, Title FROM shows', engine)

In [133]:
shows_df_amazon.loc[:, 'cast'] = shows_df_amazon['cast'].str.split(',')
shows_df_amazon = shows_df_amazon.explode('cast').reset_index(drop=True)
shows_df_amazon.loc[:, 'cast'] = shows_df_amazon['cast'].str.strip()

In [134]:
casting_mapping = dict(zip(casting_mapping_df.Name, casting_mapping_df.ID))
shows_df_amazon['ID_Cast'] = shows_df_amazon['cast'].map(casting_mapping)

shows_df_amazon = shows_df_amazon.dropna(subset=['ID_Cast'])
shows_df_amazon['ID_Cast'] = shows_df_amazon['ID_Cast'].astype(int)

In [135]:
type_mapping = dict(zip(type_mapping_df.Type, type_mapping_df.ID))
shows_df_amazon['ID_Type'] = shows_df_amazon['type'].map(type_mapping)

In [136]:
shows_df_amazon['director'] = shows_df_amazon['director'].str.split(',')
shows_df_amazon = shows_df_amazon.explode('director').reset_index(drop=True)

In [137]:
director_mapping = dict(zip(director_mapping_df.Name, director_mapping_df.ID))
shows_df_amazon['ID_Director'] = shows_df_amazon['director'].map(director_mapping)

shows_df_amazon = shows_df_amazon.dropna(subset=['ID_Director'])
shows_df_amazon['ID_Director'] = shows_df_amazon['ID_Director'].astype(int)

In [138]:
show_mapping = dict(zip(show_mapping_df.Title, show_mapping_df.ID))
shows_df_amazon['ID_Show'] = shows_df_amazon['title'].map(show_mapping)
shows_df_amazon['ID_Show'] = shows_df_amazon['ID_Show'].astype(int)

In [ ]:
shows_df_amazon['ID_Platform'] = 1

In [141]:
shows_df_amazon = shows_df_amazon.drop(['title', 'type', 'cast', 'director'], axis=1)

In [ ]:
shows_df_amazon.head()

In [143]:
#shows_df_amazon.to_sql('full_show', con=engine, if_exists='append', index=False)

37440

In [144]:
shows_df_netflix = df_netflix[['title', 'type', 'cast', 'director']]

In [145]:
type_mapping_df = pd.read_sql('SELECT ID, Type FROM category', engine)
casting_mapping_df = pd.read_sql('SELECT ID, Name FROM casting', engine)
director_mapping_df = pd.read_sql('SELECT ID, Name FROM director', engine)
show_mapping_df = pd.read_sql('SELECT ID, Title FROM shows', engine)

In [146]:
shows_df_netflix.loc[:, 'cast'] = shows_df_netflix['cast'].str.split(',')
shows_df_netflix = shows_df_netflix.explode('cast').reset_index(drop=True)
shows_df_netflix.loc[:, 'cast'] = shows_df_netflix['cast'].str.strip()

In [147]:
casting_mapping = dict(zip(casting_mapping_df.Name, casting_mapping_df.ID))
shows_df_netflix['ID_Cast'] = shows_df_netflix['cast'].map(casting_mapping)

shows_df_netflix = shows_df_netflix.dropna(subset=['ID_Cast'])
shows_df_netflix['ID_Cast'] = shows_df_netflix['ID_Cast'].astype(int)

In [148]:
type_mapping = dict(zip(type_mapping_df.Type, type_mapping_df.ID))
shows_df_netflix['ID_Type'] = shows_df_netflix['type'].map(type_mapping)

In [149]:
shows_df_netflix['director'] = shows_df_netflix['director'].str.split(',')
shows_df_netflix = shows_df_netflix.explode('director').reset_index(drop=True)

In [150]:
director_mapping = dict(zip(director_mapping_df.Name, director_mapping_df.ID))
shows_df_netflix['ID_Director'] = shows_df_netflix['director'].map(director_mapping)

shows_df_netflix = shows_df_netflix.dropna(subset=['ID_Director'])
shows_df_netflix['ID_Director'] = shows_df_netflix['ID_Director'].astype(int)

In [151]:
show_mapping = dict(zip(show_mapping_df.Title, show_mapping_df.ID))
shows_df_netflix['ID_Show'] = shows_df_netflix['title'].map(show_mapping)
shows_df_netflix['ID_Show'] = shows_df_netflix['ID_Show'].astype(int)

In [152]:
shows_df_netflix['ID_Platform'] = 2

In [153]:
shows_df_netflix = shows_df_netflix.drop(['title', 'type', 'cast', 'director'], axis=1)

In [154]:
shows_df_netflix.head()

,ID_Cast,ID_Type,ID_Director,ID_Show,ID_Platform
19,52090,2,5142,9671,2
20,52103,2,5142,9671,2
21,46517,2,5142,9671,2
22,52104,2,5142,9671,2
23,52092,2,5142,9671,2


In [155]:
#shows_df_netflix.to_sql('full_show', con=engine, if_exists='append', index=False)

45465